# TP2 - Modeling using docplex

## 1. The `docplex` python package

`DOcplex` is a python package developped by IBM &mdash; It provides easy-to-use API for IBM solvers Cplex and Cpoptimizer.

DOcplex documentation for mathematical programming can be found here: http://ibmdecisionoptimization.github.io/docplex-doc/#mathematical-programming-modeling-for-python-using-docplex-mp-docplex-mp

## 2. Solving TSP using `docplex`

### 2.1. TSP model using `docplex`

**Exercice:** Using `docplex`, create a model for the travelling salesman problem using the MTZ or Flow formulation and compare them.

In [1]:
from docplex.mp.model import Model
import numpy as np 
import tsp.data as data

distances = data.grid42
N = len(distances)

tsp = Model("TSP")
tsp.log_output = True

X = np.array([tsp.binary_var_list(N, name=f"x{i}") for i in range(N)])

tsp.add_constraints((sum(x[i,:])-x[i,i] == 1) for i in range(N))
tsp.add_constraints((sum(x[:,i])-x[i,i] == 1) for i in range(N))

tsp.minimize(sum(sum(np.multiply(x,distances))))
solution = tsp.solve()
print("z* =", solution.objective_value)

/home/lenezet/.local/lib/python3.7/site-packages/tsp/core.py:18: UserWarning: Missing netCDF4 library. Some functionality will be limited.
  warnings.warn("Missing netCDF4 library. Some functionality will be limited.")
/home/lenezet/.local/lib/python3.7/site-packages/tsp/readers.py:10: UserWarning: Missing netCDF4 library. Some functionality will be limited.
  warnings.warn("Missing netCDF4 library. Some functionality will be limited.")


AttributeError: module 'tsp.data' has no attribute 'grid42'

The largest set of distances contains 42 nodes, and should be easily solved by `docplex`.

### 2.2. Generating random TSP instances

**Question:** What method could you implement to generate a realistic set of distances for $n$ customers?

**Exercice:** Implement the method proposed above and test it.

In [10]:
#to generate a realistic set of distances for n customer we could create
# a matrix of n * n that represent the distances for n customers

import numpy as np
import scipy.spatial.distance


def generate_distances(n: int):
    dist = np.zeros((n,n))
    
    for i in range(n):
        for y in range(n):
            if (i != y):
                #on choisis une distance aléatoire
                distance = np.random.randint(1,200)
                #on remplis la matrice
                dist[i,y] = distance
                dist[y,i] = distance
    return dist
                


from docplex.mp.model import Model

distances = generate_distances(50)
print(distances)

N = len(distances)

tsp = Model("TSP")
tsp.log_output = True

X = np.array([tsp.binary_var_list(N, name=f"x{i}") for i in range(N)])

tsp.add_constraints((sum(X[i,:])-X[i,i] == 1) for i in range(N))
tsp.add_constraints((sum(X[:,i])-X[i,i] == 1) for i in range(N))

tsp.minimize(sum(sum(np.multiply(X,distances))))

solution = tsp.solve()
print("z* =", solution.objective_value)

[[  0. 109.  50. ... 157. 166. 126.]
 [109.   0. 190. ... 137. 178.  98.]
 [ 50. 190.   0. ... 168. 184. 135.]
 ...
 [157. 137. 168. ...   0.  77.  61.]
 [166. 178. 184. ...  77.   0. 192.]
 [126.  98. 135. ...  61. 192.   0.]]
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 4910.000000 after 0.00 sec. (0.19 ticks)
Found incumbent of value 590.000000 after 0.00 sec. (0.26 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 50 columns.
Reduced MIP has 100 rows, 2450 columns, and 4900 nonzeros.
Reduced MIP has 2450 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.14 ticks)
Tried aggregator 1 time.
Reduced MIP has 100 rows, 2450 columns, and 4900 nonzeros.
Reduced MIP has 2450 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (4.09 ticks)
Probing time = 0.01 sec. (2.93 ticks)
Clique table members: 100.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel 

## 3. Solving Warehouse Allocation using Benders decomposition with `docplex`

### 3.1. The warehouse problem

A company needs to supply a set of $n$ clients and needs to open new warehouses (from a
set of $m$ possible warehouses).
Opening a warehouse $j$ costs $f_j$ and supplying a client $i$ from a warehouse $j$ costs $c_{ij}$ per supply unit.
Which warehouses should be opened in order to satisfy all clients while minimizing the total cost?

### 3.2. Solving the warehouse problem with a single ILP

- $y_{j} = 1$ if and only if warehouse $j$ is opened.
- $x_{ij}$ is the fraction supplied from warehouse $j$ to customer $i$.

$
\begin{align}
  \text{min.} \quad & \sum_{i=1}^{n} \sum_{j=1}^{m} c_{ij} x_{ij} + \sum_{j=1}^{m} f_{j} y_{j} & \\
  \text{s.t.} \quad & \sum_{j=1}^{m} x_{ij} = 1, & \forall i \in\{1,\ldots,n\}\\
                    & x_{ij} \leq y_{j}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                    & y_{j} \in \left\{0,~1\right\}, & \forall j \in \left\{1,~\ldots,~m\right\}\\
                    & x_{ij} \geq 0, & \forall i \in \left\{1,~\ldots,~n\right\}, \forall j \in \left\{1,~\ldots,~m\right\}
\end{align}
$


**Exercice:** Implement the ILP model for the warehouse allocation problem and test it on the given instance.

In [11]:
from docplex.mp.model import Model
#import numpy as np

# We will start with a small instances with 3 warehouses and 3 clients:
M = 3
N = 3


# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

wa = Model("Warehouse Allocation")
wa.log_output = True

x = [wa.continuous_var_list(M, name=f"x{i}") for i in range(N)]
y = wa.binary_var_list(M)


for i in range(N):
    wa.add_constraint(sum(x[i][j] for j in range(M)) == 1)
    for j in range(M):
        wa.add_constraint(x[i][j] <= y[j])
        wa.add_constraint(x[i][j] >= 0)

wa.set_objective("min",  sum(c[i][j]*x[i][j] for i in range(N) for j in range(M)) + sum(f[j]*y[j] for j in range(M)))

solution = wa.solve()
print("z* =", solution.objective_value)

CPXPARAM_Read_DataCheck                          1
Found incumbent of value 40.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 9 rows and 0 columns.
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 32 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                           40.0000        0.000

### 3.3. Benders' decomposition for the Warehouse Allocation problem

We are going to use Benders' decomposition to solve the Warehouse Allocation problem. 

#### Dual subproblem

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

#### Master problem

$
\begin{align*}
  \text{min.} \quad & \sum_{j=1}^{m} f_j y_j + z & \\
  \text{s.t.} \quad & z \geq \sum_{i=1}^{n}v_i^p - \sum_{i=1}^{n} \sum_{j=1}^{m} u_{ij}^p y_j, & \forall p\in l_1\\
                    & 0 \geq \sum_{i=1}^{n}v_i^r - \sum_{i=1}^{n} \sum_{j=1}^{n} u_{ij}^r y_j, & \forall r\in l_2\\
                    & y_{j} \in\{0,1\}, & \forall j\in\{1,\ldots,m\}
\end{align*}
$

**Exercice:** Implement the method `create_master_problem` that creates the initial master problem (without feasibility or optimality constraints) for the warehouse allocation problem.

<div class="alert alert-info alert-block">

You can use `print(m.export_as_lp_string())` to display a textual representation of a `docplex` model `m`.
    
</div>

In [18]:
from docplex.mp.model import Model
from docplex.mp.linear import Var
from docplex.mp.constr import AbstractConstraint
from typing import List, Sequence, Tuple


def create_master_problem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]]
) -> Tuple[Model, Var, Sequence[Var]]:
    """
    Creates the initial Benders master problem for the Warehouse Allocation problem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.

    Returns:
        A 3-tuple containing the docplex problem, the z variable and the y variables.
    """

    wa = Model("Warehouse Allocation - Benders master problem")
    z = wa.continuous_var()
    y = wa.binary_var_list(N)
    
    wa.set_objective("min", sum(f[j]*y[j] for j in range(M)) + z)

    return wa, z, y


# Check your method:
wa, z, y = create_master_problem(N, M, f, c)
print(wa.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders master problem

Minimize
 obj: x1 + 20 x2 + 20 x3 + 20 x4
Subject To

Bounds
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1

Binaries
 x2 x3 x4
End



**Exercice:** Implement the method `add_optimality_constraints` that add optimality constraints to the Benders master problem. 

In [19]:
def add_optimality_constraint(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        wa: The Benders master problem (docplex.mp.model.Model).
        z: The z variable of the master problem.
        y: The y variables of the master problem.
        v: The optimal values for the v variables of the Benders dual subproblem.
        u: The optimal values for the u variables of the Benders dual subproblem.

    Return: The optimality constraint added.
    """
    constr = wa.add_constraint(z >= (sum(v) - sum(sum(np.multiply(u,y)))))   
    return constr

**Exercice:** Implement the method `add_feasibility_constraints` that add feasibility constraints to the Benders master problem. 

In [20]:
def add_feasibility_constraints(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
      - N: Number of clients.
      - M: Number of warehouses.
      - wa: The Benders master problem (docplex.mp.model.Model).
      - z: The z variable of the master problem.
      - y: The y variables of the master problem.
      - v: The extreme rays for the v variables of the Benders dual subproblem.
      - u: The extreme rays for the u variables of the Benders dual subproblem.

    Returns:
        The feasibility constraint added.
    """
    constr = wa.add_constraint(sum(v) - sum(sum(np.multiply(u,y))) <= 0)
    return constr

**Exercice:** Implement the method `create_dual_subproblem` that, given a solution `y` of the master problem, create the corresponding Benders dual subproblem.

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

In [21]:
from docplex.mp.model import Model


def create_dual_subproblem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]], y: Sequence[int]
) -> Tuple[Model, Sequence[Var], Sequence[Sequence[Var]]]:
    """
    Creates a Benders dual subproblem for the Warehouse Allocation problem corresponding
    to the given master solution.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.
        y: Values of the y variables from the Benders master problem.

    Returns:
        A 3-tuple containing the docplex problem, the v variable and the u variables.
    """

    dsp = Model("Warehouse Allocation - Benders dual subproblem")

    # We disable pre-solve to be able to retrieve a meaningful status in the main
    # algorithm:
    dsp.parameters.preprocessing.presolve.set(0)

    u = np.array([dsp.continuous_var_list(M, name=f"u{i}") for i in range(N)])
    v = np.array(dsp.continuous_var_list(M))
    
    for i in range(N):
        for j in range(M):
            dsp.add_constraint(u[i][j] >= 0)
            dsp.add_constraint(v[i] - u[i][j] <= c[i][j])
            

    dsp.set_objective("max", sum (v) - sum(sum(np.multiply(y,u))))
    
    return dsp, v, u


# Check your method (assuming y = [1 1 1 ... 1]):
dsp, v, u = create_dual_subproblem(N, M, f, c, [1] * M)
print(dsp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders dual subproblem

Maximize
 obj: - u0_0 - u0_1 - u0_2 - u1_0 - u1_1 - u1_2 - u2_0 - u2_1 - u2_2 + x10 + x11
      + x12
Subject To
 c1: u0_0 >= 0
 c2: - u0_0 + x10 <= 15
 c3: u0_1 >= 0
 c4: - u0_1 + x10 <= 1
 c5: u0_2 >= 0
 c6: - u0_2 + x10 <= 2
 c7: u1_0 >= 0
 c8: - u1_0 + x11 <= 1
 c9: u1_1 >= 0
 c10: - u1_1 + x11 <= 16
 c11: u1_2 >= 0
 c12: - u1_2 + x11 <= 3
 c13: u2_0 >= 0
 c14: - u2_0 + x12 <= 4
 c15: u2_1 >= 0
 c16: - u2_1 + x12 <= 1
 c17: u2_2 >= 0
 c18: - u2_2 + x12 <= 17

Bounds
End



**Exercice:** Using the methods you implemented, write the Benders decomposition algorithm for the warehouse allocation problem.

<div class="alert alert-block alert-info">

The `get_extreme_rays` function can be used to retrieve the extreme rays associated with an unbounded solution of the dual subproblem.
    
</div>

<div class="alert alert-block alert-info">
    
You can use `model.get_solve_status()` to obtain the status of the resolution and compare it to members of `JobSolveStatus`:
    
```python
if model.get_solve_status() == JobSolveStatus.OPTIMAL_SOLUTION:
    pass
```
    
</div>

In [ ]:
from docplex.mp.model import Model
from docplex.util.status import JobSolveStatus


def get_extreme_rays(
    N: int, M: int, model: Model, v: Sequence[Var], u: Sequence[Sequence[Var]]
) -> Tuple[Sequence[float], Sequence[Sequence[float]]]:
    """
    Retrieves the extreme rays associated to the dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        model: The Benders dual subproblem model (docplex.mp.model.Model).
        v: 1D array containing the v variables of the subproblem.
        u: Either a 2D array of a tuple-index dictionary containing the u variables
            of the subproblem.

    Returns:
        A 2-tuple containing the list of extreme rays correspondig to v,
        and the 2D-list of extreme rays corresponding to u.
    """
    ray = model.get_engine().get_cplex().solution.advanced.get_ray()

    if isinstance(u, dict):

        def get_uij(i, j):
            return u[i, j]

    else:

        def get_uij(i, j):
            return u[i][j]

    return (
        [ray[v[i].index] for i in range(N)],
        [[ray[get_uij(i, j).index] for j in range(M)] for i in range(N)],
    )


# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

# We stop iterating if the new solution is less than epsilon
# better than the previous one:
epsilon = 1e-6

wa, z, y = create_master_problem(N, M, f, c)

n = 0
while True:

    # Print iteration:
    n = n + 1
    print("Iteration {}".format(n))
    
    solved = wa.solve
    
    

    ...  # TODO

print("Done.")

### 3.4. Generating instances for the Warehouse Allocation problem

**Exercice:** Using the TSP instances contained in `tsp.data` or the `generate_distances` method, create instances for the warehouse allocation problem with randomized opening costs.

<div class="alert alert-block alert-danger"></div>